In [1]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
from sklearn.preprocessing import MinMaxScaler

# Load data
df_madrid = pd.read_csv("madrid.csv")
df_cataluna = pd.read_csv("cataluna.csv")
geo_madrid = gpd.read_file("municipalities_madrid.geojson")
geo_cataluna = gpd.read_file("municipalities_cataluna.geojson")

df_madrid["municipality"] = df_madrid["municipality"].astype(str)
df_cataluna["municipality"] = df_cataluna["municipality"].astype(str).str.zfill(5)
geo_madrid["municipality"] = geo_madrid["municipality"].astype(str)
geo_cataluna["municipality"] = geo_cataluna["municipality"].astype(str)

# INITIALIZE APP 
app = Dash(__name__)

# === LAYOUT ===
app.layout = html.Div([
    html.Label("Select the province:"),
    dcc.Dropdown(
        id="region",
        options=[
            {"label": "Madrid", "value": "madrid"},
            {"label": "Cataluña", "value": "cataluna"}
        ],
        value="madrid"
    ),

    html.H1("Interactive map: Zone Score per Municipality"),

    html.Div([
        html.H3("Weights: Sociodemografic variables", style={"marginTop": 30}),

        html.Label("High"),
        dcc.Slider(id="w_high", min=0, max=1, step=0.05, value=0.2),

        html.Label("High Ratio"),
        dcc.Slider(id="w_high_ratio", min=0, max=1, step=0.05, value=0.2),

        html.Label("Professionals"),
        dcc.Slider(id="w_pro", min=0, max=1, step=0.05, value=0.2),

        html.Label("Professionals Ratio"),
        dcc.Slider(id="w_pro_ratio", min=0, max=1, step=0.05, value=0.2),

        html.Label("Europeans"),
        dcc.Slider(id="w_eur", min=0, max=1, step=0.05, value=0.1),

        html.Label("Europeans Ratio"),
        dcc.Slider(id="w_eur_ratio", min=0, max=1, step=0.05, value=0.1),

        html.H3("Weights: Business Variables"),

        html.Label("N. of New Companies"),
        dcc.Slider(id="w_new_comp", min=0, max=1, step=0.05, value=0.1),

        html.Label("Growth Rate"),
        dcc.Slider(id="w_growth", min=0, max=1, step=0.05, value=0.1),

        html.Label("N. of Medium and Large Companies"),
        dcc.Slider(id="w_ml", min=0, max=1, step=0.05, value=0.2),

        html.Label("N of High Profit Companies"),
        dcc.Slider(id="w_hp", min=0, max=1, step=0.05, value=0.1),

        html.Label("Medium and Large Companies Ratio"),
        dcc.Slider(id="w_size", min=0, max=1, step=0.05, value=0.2),

        html.Label("High Profit Companies Ratio"),
        dcc.Slider(id="w_profit", min=0, max=1, step=0.05, value=0.1),

        html.H3("Weights: Zone Score (sum must be 1)"),

        html.Label("Sociodemographic Score"),
        dcc.Slider(id="w_sociozone", min=0, max=1, step=0.05, value=0.45),

        html.Label("Business Score"),
        dcc.Slider(id="w_businesszone", min=0, max=1, step=0.05, value=0.45),

        html.Label("Competitor Score"),
        dcc.Slider(id="w_comp", min=0, max=1, step=0.05, value=0.1),
    ]),

    html.Div(id="warning-msg", style={"color": "red", "fontWeight": "bold", "marginTop": 20}),

    dcc.Graph(id="choropleth")
])

# === CALLBACK ===
@app.callback(
    [Output("choropleth", "figure"),
     Output("warning-msg", "children")],
    [
        Input("region", "value"),
        Input("w_high", "value"),
        Input("w_high_ratio", "value"),
        Input("w_pro", "value"),
        Input("w_pro_ratio", "value"),
        Input("w_eur", "value"),
        Input("w_eur_ratio", "value"),
        Input("w_new_comp", "value"),
        Input("w_growth", "value"),
        Input("w_ml", "value"),
        Input("w_hp", "value"),
        Input("w_size", "value"),
        Input("w_profit", "value"),
        Input("w_sociozone", "value"),
        Input("w_businesszone", "value"),
        Input("w_comp", "value")
    ]
)
def update_map(region, w_high, w_high_ratio, w_pro, w_pro_ratio, w_eur, w_eur_ratio,
               w_new_comp, w_growth, w_ml, w_hp, w_size, w_profit,
               w_sociozone, w_businesszone, w_comp):

    if region == "madrid":
        df = df_madrid.copy()
        geo = geo_madrid.copy()
    else:
        df = df_cataluna.copy()
        geo = geo_cataluna.copy()

    total_zone_weight = w_sociozone + w_businesszone + w_comp
    warning = ""

    if abs(total_zone_weight - 1.0) > 0.01:
        warning = f"The sum of the Zone Score weights is {total_zone_weight:.2f}, must be 1.0."

    df["sociodemo_Score"] = (
        w_high * df["high_Score"] +
        w_high_ratio * df["high_ratio_Score"] +
        w_pro * df["professionals_Score"] +
        w_pro_ratio * df["professionals_ratio_Score"] +
        w_eur * df["europeans_Score"] +
        w_eur_ratio * df["europeans_ratio_Score"]
    )

    df["Business_Score"] = (
        w_new_comp * df["NumNewCompanies_10S_Score"] +
        w_growth * df["growth_rate_10S_Score"] +
        w_ml * df["NumCompanies_ML_10S_Score"] +
        w_hp * df["NumCompanies_HP_10S_Score"] +
        w_size * df["companies_size_ratio_10S_Score"] +
        w_profit * df["companies_profit_ratio_10S_Score"]
    )

    scaler = MinMaxScaler()
    df[["sociodemo_Score_norm", "Business_Score_norm", "Competitor_Score_norm"]] = scaler.fit_transform(
        df[["sociodemo_Score", "Business_Score", "Competitor_Score"]]
    )

    df["Zone_Score"] = (
        w_sociozone * df["sociodemo_Score_norm"] +
        w_businesszone * df["Business_Score_norm"] +
        w_comp * df["Competitor_Score_norm"]
    )

    gdf = geo.merge(df[["municipality", "Zone_Score", "municipality_name"]], on="municipality", how="left")

    fig = px.choropleth_mapbox(
        gdf,
        geojson=gdf.geometry,
        locations=gdf.index,
        color="Zone_Score",
        mapbox_style="carto-positron",
        center={"lat": 41.5, "lon": 1.5} if region == "cataluna" else {"lat": 40.4168, "lon": -3.7038},
        zoom=7,
        opacity=0.6,
        labels={"Zone_Score": "Zone Score"},
        hover_name="municipality_name",
        hover_data={"municipality": True, "Zone_Score": ":.2f"}
    )

    return fig, warning

# === CORRER APP ===
if __name__ == "__main__":
    app.run(debug=True, port=8051)
